<a href="https://colab.research.google.com/github/kundana12345/52.LASAnix-IPC-queue/blob/main/Kundana_Unit7ExercisesSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fitting Curves: Concepts

What you'll do:

- Answer questions about what a GP is, and its relationship to GLMs and splines.
- Practice applying each of: polynomial modeling, b splines, and GPs
- You'll get a chance to read about and try to comprehend a more standard implementation of a GP.

Have fun!

In [9]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import arviz as az
import xarray as xr
import pymc as pm
from scipy.interpolate import PchipInterpolator

In [19]:
# B-spline model
model_bspline = bmb.Model("weight ~ bs(height, df=10)", howell, family="gaussian")
idata_bspline = model_bspline.fit()
# Explicitly sample posterior predictive
with model_bspline.backend.model:
    ppc_bspline = pm.sample_posterior_predictive(idata_bspline.posterior, var_names=["weight"])
    idata_bspline.posterior_predictive = ppc_bspline.posterior_predictive

Output()

TypeError: sample_posterior_predictive() got an unexpected keyword argument 'extend_idata'

In [12]:
def plot_ppc_scatter(idata, title, x_data, y_data, x_label, y_label):
    # Reshape posterior predictive samples to (n_samples, n_data_points)
    pp_weight = idata.posterior_predictive['weight'].values.reshape(-1, len(x_data))

    # Sort data by x_data for smooth plotting of mean and HDI
    sorted_idx = np.argsort(x_data)
    x_data_sorted = x_data[sorted_idx]
    y_data_sorted = y_data[sorted_idx]
    pp_weight_sorted = pp_weight[:, sorted_idx]

    # Calculate mean and 95% HDI of posterior predictive
    mean_pp = np.mean(pp_weight_sorted, axis=0)
    hdi_pp = az.hdi(pp_weight_sorted, hdi_prob=0.95)

    plt.figure(figsize=(10, 6))
    plt.scatter(x_data, y_data, alpha=0.5, label='Original Data')
    plt.plot(x_data_sorted, mean_pp, color='red', label='Posterior Predictive Mean')
    plt.fill_between(x_data_sorted, hdi_pp[:, 0], hdi_pp[:, 1], color='red', alpha=0.2, label='95% HDI')
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.grid(True)
    plt.show()

# Convert howell height and weight to numpy arrays for plotting
height_np = howell['height'].values
weight_np = howell['weight'].values

# Plot for Polynomial Model
plot_ppc_scatter(idata_poly, "Polynomial Model PPC (Scatter)", height_np, weight_np, "Height", "Weight")

# Plot for B-spline Model
plot_ppc_scatter(idata_bspline, "B-spline Model PPC (Scatter)", height_np, weight_np, "Height", "Weight")

# Plot for Gaussian Process Model
plot_ppc_scatter(idata_gp, "Gaussian Process Model PPC (Scatter)", height_np, weight_np, "Height", "Weight")

AttributeError: 'InferenceData' object has no attribute 'posterior_predictive'

**Task1**:

Why would you ever want to include a polynomial element in a model you built? What's the benefit of using polynomials to model?

When your data is really complicated, having a polynomial element can be helpful.

**Task2**:

Why would you ever NOT want to include a polynomial element in a model you built?

Polynomials can lead to overfitting.

**Task3**:

What's the point of using b splines?

It helps us used polynomial elements while reducing the overfitting that happens. It splits the data into different sections and tries to predit each section by itself.

**Task4**:

Describe what a Gaussian Process is, in your own words. *Don't worry about being correct, just try to explain it to yourself*. I will not grade this question for accuracy.

It has multiple splines that choose their own knots and it sort of splits the data and uses splines and knots to form a model for each seperate part.

**Task5**:

Fit three models to the howell data (from Unit5ExercisesSF): polynomial, b splines, and Gaussian Process.

Plot the posterior predictive check on a scatter plot, as is standard/required.

Hint: Distributional models (variable variance) work better on the howell data.


In [8]:
howell = pd.read_csv('https://raw.githubusercontent.com/thedarredondo/data-science-fundamentals/main/Data/howell.csv')

In [4]:
howell

,height,weight,age,male
0,151.765,47.825606,63.0,1
1,139.700,36.485807,63.0,0
2,136.525,31.864838,65.0,0
3,156.845,53.041914,41.0,1
4,145.415,41.276872,51.0,0
...,...,...,...,...
539,145.415,31.127751,17.0,1
540,162.560,52.163080,31.0,1
541,156.210,54.062497,21.0,0
542,71.120,8.051258,0.0,1


In [6]:
!pip install preliz
!pip uninstall bambi arviz_stats -y
!pip install bambi==0.16.0

Found existing installation: bambi 0.17.2
Uninstalling bambi-0.17.2:
  Successfully uninstalled bambi-0.17.2
Found existing installation: arviz-stats 0.8.0
Uninstalling arviz-stats-0.8.0:
  Successfully uninstalled arviz-stats-0.8.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.3 MB/s eta 0:00:00


In [7]:
import bambi as bmb

In [ ]:
#used Gemini

In [18]:
model_poly = bmb.Model("weight ~ poly(height, degree=2)", howell, family="gaussian")
idata_poly = model_poly.fit()
# Explicitly sample posterior predictive
with model_poly.backend.model:
    ppc_poly = pm.sample_posterior_predictive(idata_poly.posterior, var_names=["weight"])
    idata_poly.posterior_predictive = ppc_poly.posterior_predictive

Output()

TypeError: sample_posterior_predictive() got an unexpected keyword argument 'extend_idata'

In [20]:
import pymc as pm
import pytensor.tensor as pt

# Prepare data for PyMC GP
X_gp = howell["height"].values[:, None] # Reshape to 2D for GP
y_gp = howell["weight"].values

with pm.Model() as gp_model_pymc:
    # GP parameters
    ℓ = pm.HalfCauchy("ℓ", beta=5)
    η = pm.HalfCauchy("η", beta=5)
    sigma = pm.HalfCauchy("sigma", beta=2)

    # Covariance function: Exponential Quadratic (Radial Basis Function)
    cov = η**2 * pm.gp.cov.ExpQuad(input_dim=1, ls=ℓ)

    # Mean function: Constant mean of the observed data
    mean_func = pm.gp.mean.Constant(y_gp.mean())

    # Gaussian Process prior
    gp_latent = pm.gp.Latent(cov_func=cov, mean_func=mean_func)

    # Latent function f, evaluated at the observed X_gp points
    f = gp_latent.prior("f", X=X_gp)

    # Likelihood: Normal distribution for the observed weight
    y_obs = pm.Normal("y_obs", mu=f, sigma=sigma, observed=y_gp)

    # Sampling
    idata_gp = pm.sample(2000, tune=2000, random_seed=42)

    # Posterior predictive sampling at observed points X_gp
    # Using conditional distribution for new data, here Xnew is the same as X_gp
    # pm.sample_posterior_predictive returns an InferenceData object
    ppc_gp = pm.sample_posterior_predictive(idata_gp, var_names=["y_obs"])
    idata_gp.posterior_predictive = ppc_gp.posterior_predictive

    # Rename 'y_obs' in posterior_predictive to 'weight' to match plot_ppc_scatter function
    idata_gp.posterior_predictive["weight"] = idata_gp.posterior_predictive["y_obs"]
    del idata_gp.posterior_predictive["y_obs"]

AttributeError: module 'bambi' has no attribute 'GaussianProcessTerm'

**Task6**:

Read the article on the pymc website about GP implementation on the Mauna Loa CO$_{2}$ data combined with CO$_{2}$ ice core data from the south pole.
[Link here.](https://www.pymc.io/projects/examples/en/latest/gaussian_processes/GP-MaunaLoa2.html)

Write down one thing you learned about GPs from reading the article.

Note: You probably won't understand much in this article--I had to read it about five times before I figured out what was going on. The points of this task are to: hammer home that good GP implementations are extremely technical, and increasure your exposure to the kinds of problems traditional GPs are good at solving.

Covariance measures how much two variables move together.

**Task7**:

Describe your favorite graph from the article in the previous task with as much technical detail as you can muster.

Explain why its your favorite.